In [1]:
!pip install transformers torch pandas scikit-learn

In [6]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset

In [2]:
!pip3 install datasets

  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.1
    Uninstalling tqdm-4.64.1:
      Successfully uninstalled tqdm-4.64.1
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deeppavlov 1.7.0 requires filelock<3.10.0,>=3.0.0, but you have filelock 3.16.1 which is incompatible.
deeppavlov 1.7.0 requires tqdm<4.65.0,>=4.42.0, but you have tqdm 4.66.5 which is incompatible.


In [29]:
data = pd.read_csv('intent_dataset.csv')

In [30]:
# Map labels to integers if they aren't already
label_mapping = {label: idx for idx, label in enumerate(sorted(data['Label'].unique()))}
data['Label'] = data['Label'].map(label_mapping)
num_labels = len(label_mapping)


In [31]:
# Load your dataset
data = pd.read_csv('intent_dataset.csv')

# Assume 'Text' is your feature and 'Label' is your encoded target
# Let's inspect the dataset structure
print(data.head())


                                                Text  Label
0  I need to change the delivery address for my o...      0
1               Please cancel my order number 12345.      0
2  My gift card code isn't working when I try to ...      1
3  Do you have any discount codes available right...      1
4  One of the items in my order is missing, can y...      1


In [32]:
# Check class distribution
print(data['Label'].value_counts())


4     4
1     3
3     3
0     2
2     2
11    2
12    2
Name: Label, dtype: int64


In [33]:
# Check class distribution
class_counts = data['Label'].value_counts()

# Filter out classes with only 1 sample
filtered_data = data[data['Label'].isin(class_counts[class_counts > 1].index)]

# Features and target
X = filtered_data['Text']
y = filtered_data['Label']

# Proceed with train_test_split using stratified sampling
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.4,
    random_state=42,
    stratify=y  # Ensures proportional distribution of classes
)


In [34]:
print("Training set class distribution:\n", y_train.value_counts())
print("Test set class distribution:\n", y_test.value_counts())

Training set class distribution:
 4     2
1     2
3     2
0     1
12    1
11    1
2     1
Name: Label, dtype: int64
Test set class distribution:
 4     2
2     1
12    1
11    1
0     1
3     1
1     1
Name: Label, dtype: int64


In [35]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Compute class weights based on the training set
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),  # Unique classes in the training set
    y=y_train  # Labels
)

# Map the class weights to their respective labels
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

print("Class Weights: ", class_weight_dict)


Class Weights:  {0: 1.4285714285714286, 1: 0.7142857142857143, 2: 1.4285714285714286, 3: 0.7142857142857143, 4: 0.7142857142857143, 5: 1.4285714285714286, 6: 1.4285714285714286}


In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

# Create a pipeline with a Tfidf vectorizer and Logistic Regression model
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Convert text to features
    ('classifier', LogisticRegression(class_weight=class_weight_dict))  # Model with class weights
])

# Train the model
pipeline.fit(X_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


ValueError: The classes, [11, 12], are not in class_weight